This Jupyter notebook is for the [binary] text classifier on the bc-dataset.csv file.

There will be two (2) implementations.
1. Representing input as a bag of words.
2. Word vectors (can be pre-trained word embeddings).

The dataset split will be <>.
Evaluation measures will be:
1. Precision.
2. Recall.
3. F-Measure.